** We are going to be working through understanding and interacting with the PCP Plugin framework.  The SamplePlugin is copied below from the prior SamplePlugin notebook. **

** The two core functions will be to: **
* Request jobs
* Respond with job output

** The start function should be an infinite loop handling these two processes. **
** The functionality will need to be a list of valid commands. If you have built the commands for your plugin, from the Introduction.ipynb, copy and paste these commands into this notebook. **
** WARNING:  The framework may terminate your thread, but it will call your _stop() function when the container is stopped. **

** The sample command is pasted below and the command variable (cmdWget) is pasted into your SamplePlugin Class to demonstrate building the command(s) into your custom plugin. **

In [2]:
from controller_plugin import ControllerPlugin
from os import environ
environ["PORT"] = "10000" 

In [3]:
cmdWget = {"CommandName" : "wget",
          "Tooltip" : "Retrieves raw webpage files from targeted systems",
          "Output" : True,
          "Inputs" : [{"Name" : "Target IP Address",
                     "Type" : "url",
                     "Tooltip" : "Modify the URL to the desired target IP address",
                     "Value" : "http://192.168.1.100"}]}

In [4]:
class SamplePlugin(ControllerPlugin):
    functionality =  [cmdWget]                         # Get the commands you built from the prior tutorial
    def __init__(self):
        """
        The plugin class is the last class to init,
        but Database and logger have not started yet
        Should not be using the database handles or the logger in this fn
        """
        import socket
        self.name = "SamplePlugin"
        super().__init__(self.name, self.functionality)
    def _start(self):
        """
        Start function is called by the base class and is the container entrypoint
        This function will assume the db is already started
        This function should include all calls required to run until _stop is called.

        :return: This function calls exit! when complete
        """
        #################################################################################################
        # Add Plugin specific startup routine here
        # create socket to listen for connections from the client
        self.s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.s.bind(('localhost', self.port))
        self.s.listen(1)
        pass
        ################################################################################################
        # Add plugin control loop here
        while True:
            # Assume you have a connected client for this controller
            conn, client_addr = self.s.accept() #from socket.connect
            new_job = self.request_job_for_client(client_addr, self.port)  # <dict> or None  
            if not None:  # use your plugin functionality to get this job to your connected client
                # Setting the status is optional
                self._update_job_status(job["id"], "Active")
                output_content = None # or <add code for the plugin content for this target/job (string obj)
                # Give this content back to the framework
                self.respond_output(job, output_content)
                # Setting the status is optional
                self._update_job_status(job["id"], "Done")
            sleep(3)
        
    def _stop(self):
        ###############################################################################################
        # Put cleanup code here
        self.s.close()
        self.s = None
        pass
        ###############################################################################################
        
        pass

In [5]:
SamplePlugin()